In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
 mnist_dataset, mnist_info = tfds.load(name ='mnist', with_info=True, as_supervised=True)

In [3]:
mnist_dataset

{'test': <_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [4]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

num_validations_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validations_samples = tf.cast(num_validations_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data=shuffled_train_and_validation_data.take(num_validations_samples)

train_data = shuffled_train_and_validation_data.skip(num_validations_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validations_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model 

#### Outline the model

In [5]:
inputsize = 785
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

#### Choose the optimizer and the loss function

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training

In [8]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data =(validation_inputs, validation_targets), validation_steps=50,verbose = 2)


Epoch 1/5
540/540 - 4s - loss: 0.0503 - accuracy: 0.9844 - val_loss: 0.0578 - val_accuracy: 0.9852
Epoch 2/5
540/540 - 3s - loss: 0.0399 - accuracy: 0.9880 - val_loss: 0.0670 - val_accuracy: 0.9815
Epoch 3/5
540/540 - 3s - loss: 0.0353 - accuracy: 0.9887 - val_loss: 0.0501 - val_accuracy: 0.9860
Epoch 4/5
540/540 - 3s - loss: 0.0296 - accuracy: 0.9909 - val_loss: 0.0454 - val_accuracy: 0.9880
Epoch 5/5
540/540 - 3s - loss: 0.0261 - accuracy: 0.9921 - val_loss: 0.0493 - val_accuracy: 0.9848


### Test the model

In [11]:
test_loss,test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 744ms/step - loss: 0.0949 - accuracy: 0.9753

In [15]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.53%
